# Window Operations Exercise

### Exercise

In [1]:
import sys
import random
import pathlib
import os

import findspark

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext

from apache_log_parser import ApacheAccessLog

findspark.init()

In [2]:

conf = (SparkConf().setMaster("local[4]").setAppName("log processor").set("spark.executor.memory", "2g"))

sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 15:47:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [3]:
# create DStream from text file
# Note: the spark streaming checks for any updates to this directory.
# So first, start this program, and then copy the log file access_log.log to directory location
curr = pathlib.Path().resolve()
logs_directory = os.path.join(curr / 'logs')

log_data = ssc.textFileStream(logs_directory)
access_log_dstream = log_data.map(ApacheAccessLog.parse_from_log_line).filter(lambda parsed_line: parsed_line is not None)

# ip_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, 1)) 
# ip_count = ip_dstream.reduceByKey(lambda x,y: x+y)

# ip_bytes_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, parsed_line.content_size))
# ip_bytes_sum_dstream = ip_bytes_dstream.reduceByKey(lambda x,y: x+y)

# ip_bytes_request_count_dstream = ip_count.join(ip_bytes_sum_dstream)

# ip_bytes_request_count_dstream.pprint(num = 30)

In [4]:
####### TODO: use window() to count data over a window##########################
access_logs_window = access_log_dstream.window(windowDuration = 6, slideDuration=4) 
window_counts = access_logs_window.count()
window_counts.pprint()

####### Exercise End ##########################################################

In [5]:
ssc.start() 


In [ ]:
ssc.awaitTerminationOrTimeout(timeout=200)

-------------------------------------------
Time: 2024-03-26 15:47:22
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:26
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:30
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:34
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:38
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:42
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:46
-------------------------------------------



Cannot parse logline: h194n2fls308o1033.telia.com - - [09/Mar/2004:13:49:05 -0800] "-" 408 -


-------------------------------------------
Time: 2024-03-26 15:47:50
-------------------------------------------
1545

-------------------------------------------
Time: 2024-03-26 15:47:54
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:47:58
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:02
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:06
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:10
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:14
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:18
-------------------------------------------

-------------------------------------------
Time: 2024-03-26 15:48:22
-----

## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#discretized-streams-dstreams